# Import

In [1]:
import os
import numpy as np
import random
import time
from PIL import Image
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3_arg_scope

# Build graph

In [2]:
g = tf.get_default_graph()

## Define preprocessing

In [3]:
with tf.name_scope('Preprocessing'):
    input_im = tf.placeholder(tf.uint8, shape=[None, None, None, 3], name='input_im')
    resized_im = tf.image.resize_bilinear(tf.image.convert_image_dtype(
        tf.convert_to_tensor(input_im) ,dtype=tf.float32), [299,299], name='resized_im')
    normalized_im = tf.mul(tf.sub(resized_im, 0.5), 2.0, name='normalized_im')

## Load Inception-v3 graph from slim

In [4]:
with slim.arg_scope(inception_v3_arg_scope()):
    logits, end_points = inception_v3(normalized_im, num_classes=1001, is_training=False)
slim_variables = [v for v in tf.global_variables() if v.name.startswith('InceptionV3/')]

## Create own branch

In [5]:
with tf.name_scope('Own'):
    y = tf.placeholder("float")
    x = tf.reshape(g.get_tensor_by_name('InceptionV3/Logits/SpatialSqueeze:0'),(1,1001))
    w = tf.Variable(tf.random_normal([int(x.get_shape()[-1]),300], stddev=float('1e-5')), name='weights')
    b = tf.Variable(tf.random_normal([1,300]), name='bias')
    y_pred = tf.add(tf.matmul(x, w), b, name='y_pred')
    cost = tf.reduce_sum(tf.square(y-y_pred),name='cost')
    cost_summary = tf.summary.scalar('cost', cost)

## Define training

In [6]:
with tf.name_scope('Optimization'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=float('1e-5'), name='Optimizer')
    
    variables_to_train = [w,b]
    variables_to_train.extend([v for v in tf.trainable_variables() if
                               v.name.startswith('InceptionV3/Logits') and
                               (v.name.endswith('weights:0') or v.name.endswith('biases:0'))])
    variables_to_train.extend([v for v in tf.trainable_variables() if
                               v.name.startswith('InceptionV3/Mixed_7c') and
                               (v.name.endswith('weights:0') or v.name.endswith('beta:0'))])
    
    train_step = optimizer.minimize(cost, var_list=variables_to_train, name='Train_step')

## Save graph

In [7]:
timestamp = time.strftime('%Y%m%d-%H%M', time.localtime())
tf.summary.FileWriter('./runs/'+timestamp+'/graph/', graph=g)

# Read data

In [ ]:
# Images
image_path = '../../../data/train2014/'
image_list = os.listdir(image_path)

# Caption dictionaries
vec_dict = np.load('../../../data/word2vec_train.npy').item()

# Labels
labels = open('../../../data/ilsvrc2012_labels.txt', 'r').readlines()

# Train model

In [ ]:
# Tried to follow
# https://github.com/tensorflow/models/tree/master/slim
# http://stackoverflow.com/questions/35274457/inceptionv3-and-transfer-learning-with-tensorflow/40709836#40709836
with tf.Session() as sess:
    
    # Initialize variables
    saver = tf.train.Saver(var_list=slim_variables)
    saver.restore(sess, "../../../data/inception-2016-08-28/inception_v3.ckpt")
    sess.run([w.initializer, b.initializer])
    
    # Initialize writers
    train_writer = tf.summary.FileWriter('./runs/'+timestamp+'/sums/train/', flush_secs=20)
    test_writer = tf.summary.FileWriter('./runs/'+timestamp+'/sums/test/', flush_secs=20)
    
    # Train
    print('TRAINING')
    for i, im in enumerate(image_list[:2000]):
        try:
            # Choose one of the five captions randomly
            r = random.randrange(len(vec_dict[im]))
            y_temp = vec_dict[im][r].reshape((1,300))

            # Form input and feed through
            image = [np.array(Image.open(image_path + im))]
            train_writer.add_summary(sess.run(cost_summary,{input_im:image, y:y_temp}), i)
            sess.run(train_step,feed_dict={input_im:image, y:y_temp})
            print(i, end=' ')

    #        # Get prediction
    #        class_pred = np.argmax(sess.run(
    #            g.get_tensor_by_name('InceptionV3/Predictions/Softmax:0'),{input_im:image}))
    #        print(labels[class_pred])

            # Evaluate every 10th step
            if i % 10 == 0:
                test_image = [np.array(Image.open(image_path + image_list[-1]))]
                y_test = vec_dict[image_list[-1]][1].reshape((1,300))
                test_writer.add_summary(sess.run(cost_summary,{input_im:test_image, y:y_test}), i)
        except (IndexError, ValueError):
            print('-/{}'.format(i), end=' ')
            
    # Save
    save_path = saver.save(sess, './runs/'+timestamp+'/checkpoint/model.ckpt')
    print('\n\nTrained model saved to {}'.format(save_path))

TRAINING
0 1 2 3 4 5 6 7 8 9 10 11 12 -/13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 -/109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 -/181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273